# **1) Import the Modules**

Modules are code libraries that contain a set of ready-to-use functions.

* The `ee` module allows developers to interact with Google Earth Engine using the Python programming language.
* The `os` module provides functions to perform tasks such as file and directory operations, process management, and environment variable manipulation.
The `math` module provides a collection of mathematical functions and constants for performing mathematical operations.
* The `geemap` module allows interactive analysis and visualization of GEE datasets in a Jupyter environment.
* The `matplotlib.pyplot` module provides a collection of functions for creating and customizing plots, diagrams and visualizations.
* The `google.colab` module provides access to some of the unique features and functionality of Google Colab.

In [1]:
!pip install geemap
!pip install ffmpeg-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.8 MB/s eta 0:00:00


In [2]:
import ee
import os
import math
import geemap
import ffmpeg

import matplotlib.pyplot as plt

from google.colab import drive

***Update the geemap package***

If you encounter errors with this notebook, please remove the comment in the line below to update the geemap package to the latest version from GitHub. Restart the Kernel (Menu -> Kernel -> Restart) to take effect.

In [3]:
# geemap.update_package()

# **2) Authentication Procedure**

This section provides instructions for setting up the Google Earth Engine Python API on Colab and for setting up Google Drive on Colab. These steps should be performed each time you start/restart/rollback a Colab session.

## **2.1) GEE**

The `ee.Authenticate` function authenticates access to the Google Earth Engine servers, while the `ee.Initialize` function initializes it. After executing the following cell, the user is prompted to grant Google Earth Engine access to their Google account.

**Note:** The Earth Engine API is installed by default in Google Colaboratory.

In [5]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=txOEaugWTSXhC-jtGULxMyzy7Bysci77PRlx9dCShyQ&tc=Y0fbLIrIaJqanPAcUpI5xO2c2IeYjVJIWxmucxNAWw0&cc=J5Npm4f0yLqHZe6yKpBWIPtdN0eO4BpBLhOo50q_jHE

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXnPfR2TM0AWuEmOhQOJylv8QWI_AsSeyOmlsrDsuGjrKB8efITFvOk

Successfully saved authorization token.


## **2.2) GD**

The `drive.mount` function allows access to specific folders of Google Drive. Granting access to Google Drive allows code running in the notebook to modify files in Google Drive.

**Note:** When using the `Mount Drive` button in the file browser, no authentication codes are required for notebooks edited only by the current user.

In [6]:
drive.mount("/content/gdrive")

Mounted at /content/gdrive


# **3) Functions**

Data Visualization

In [7]:
def compute_mean_precipitation(image):
  """
  Description:
    Applies a mean reducer to an image within a specified region.

  Args:
    image (ee.Image): The Earth Engine image on which the reducer will be applied.

  Returns:
    A GEE raster with the `mean_precipitation` property added,
    representing the mean precipitation value within the specified region.
  """
  reducedPrecipitation = image.reduceRegion(**{
    "geometry": areaOfInterest,
    "reducer": ee.Reducer.mean(),
    "scale": image.projection().nominalScale(),
    "maxPixels": 1e13
  })  \
  .get("precipitation")

  return image.set("mean_precipitation", reducedPrecipitation)


def plot_precipitation(dct: dict, destinationPath: str):
  """
  Description:
    Creates a bar plot raster of daily precipitations.

  Arguments:
    dct (dict): A dictionary of daily precipitation mm level values.
    destinationPath (str): The destination path of the bar plot.

  Returns:
    None
  """
  # Extract precipitation keys and values
  features, values = zip(*dct.items())

  # Create a figure and a set of subplots
  fig, ax = plt.subplots(figsize=(8, 6))

  # Set plot title
  plt.title("Daily Precipitation (mm)", fontsize=18, fontweight="bold", color="#595959")

  # Edit plot axes

  plt.xticks(rotation=45, ha="center", fontsize=14, fontweight="bold", color="#595959")
  plt.yticks(fontsize=14, fontweight="bold", color="#595959")

  # Add grid lines
  plt.grid(color="white", linestyle="--", linewidth=0.5)

  # Set limits
  plt.ylim(0, math.floor(max(dct.values()) + 5))

  # Set facecolor
  ax.set_facecolor("lightgray")

  # Create bar plot

  plt.bar(features, values)

  # Add bar labels
  for i, v in enumerate(values):
    plt.text(i, v, f"{v:.3f}", ha="center", va="bottom", fontsize=14)

  # Save plot
  plt.savefig(destinationPath, dpi=500, bbox_inches="tight")
  plt.close()

# **4) Parameters**

In [80]:
coordinates = [
  20.058040632927096, 38.102793131409605,
  22.899790328988313, 38.49643039052389,
  22.90055614340465,  38.512305042085146,
  22.637037745648232, 39.82427938096713,
  22.59494267298734,  39.989073630192436,
  22.59136250228799,  39.998627426959494,
  19.688106510271055, 39.606279267839795
]
acquisitions = ["2014-12-12", "2014-12-13", "2014-12-14", "2014-12-15", "2014-12-16"]

# GD paths.
destinationFolder = "/content/gdrive/MyDrive/t-h-e-s-i-s/results/emsr117/"

# **5) Configuration**

In [81]:
# `CHIRPS Daily`
chirpsConfig = {
  "name": "UCSB-CHG/CHIRPS/DAILY"
}

chirpsVisualization = {
  "min": 0,
  "max": 25,
  "palette": [
    "fff7fb", "ece7f2", "d0d1e6",
    "a6bddb", "74a9cf", "3690c0",
    "0570b0", "045a8d", "023858"
  ]
}

In [82]:
# GEE assets.
areaOfInterest = ee.Geometry.Polygon(coordinates)
ring = ee.Geometry.LinearRing(areaOfInterest.coordinates().flatten())

# **6) Data Processing**

Load, filter and process raster collection.

In [83]:
collection = ee.ImageCollection(chirpsConfig["name"])                                 \
  .map(lambda raster: raster.set("acquisition", raster.date().format("YYYY-MM-dd")))  \
  .filter(ee.Filter.inList("acquisition", acquisitions))                              \
  .map(lambda raster: raster.clip(areaOfInterest))                                    \
  .map(compute_mean_precipitation)

precipitationDates = collection.aggregate_array("acquisition")
precipitationAmounts = collection.aggregate_array("mean_precipitation")
recentPrecipitation = ee.Dictionary.fromLists(precipitationDates, precipitationAmounts).getInfo()

# **6) Data Visualization**

Create a histogram illustrating the mean precipitation levels.

In [84]:
# Construct output path.
startDate = acquisitions[0].replace("-", "_")
endDate = acquisitions[-1].replace("-", "_")
dateRange = startDate + "_to_" + endDate + ".png"
exportPath = os.path.join(destinationFolder, "precipitations", dateRange)

# Plot and export daily precipitation.
plot_precipitation(recentPrecipitation, exportPath)
print(f"Precipitation plot generated and exported to: `{exportPath}`")

Precipitation plot generated and exported to: `/content/gdrive/MyDrive/t-h-e-s-i-s/results/emsr692/magnesia/precipitations/2021_12_30_to_2022_01_03.png`


# **7) Map Visualization**

In [ ]:
Map = geemap.Map()
Map.centerObject(areaOfInterest)

rasters = collection.toList(collection.size())

for i in range(rasters.size().getInfo()):
  raster = ee.Image(rasters.get(i))
  acquisition = raster.get("acquisition").getInfo()
  Map.addLayer(raster, chirpsVisualization, acquisition)

Map.addLayer(ring, {"color": "white"}, "vectors: area")

Map

-End of Notebook-